In [11]:
import os 
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.stats.multitest as smt

In [12]:
def compile_W_to_3d_array(folder_path):
    # List all files in the folder
    file_list = os.listdir(folder_path)

    # Filter only the desired CSV files
    csv_files = [file for file in file_list if file.startswith('best.W.rank_') and file.endswith('.csv')]

    # Read and store each CSV file in a list
    dataframes = [pd.read_csv(os.path.join(folder_path, csv_file), index_col=0) for csv_file in csv_files]

    # Get the shape and inidices 
    shape = dataframes[0].shape
    index = dataframes[0].index

    # Create a 3D array of zeros with the dimensions (rows, columns, number_of_files)
    array_3d = np.zeros((shape[0], shape[1], len(dataframes)))

    # Fill the 3D array with the values from the DataFrames
    for i, df in enumerate(dataframes):
        array_3d[:, :, i] = df.values

    return index, array_3d

def find_significant_indices_fdr(three_d_array, alpha=0.05):
    rows, columns, _ = three_d_array.shape
    p_values = []

    # Perform one-sample t-tests and collect p-values
    for i in range(rows):
        for j in range(columns):
            samples = three_d_array[i, j]
            _, p_value = stats.ttest_1samp(samples, 0)
            p_values.append(p_value)
            
    # Filter out NaN values
    non_nan_indices = ~np.isnan(p_values)
    non_nan_p_values = [p_value for p_value in p_values if not np.isnan(p_value)]

    # Apply FDR correction using Benjamini-Hochberg procedure
    rejected, corrected_p_values = smt.multipletests(non_nan_p_values, alpha=alpha, method='fdr_bh')[:2]

    # Map rejected and corrected_p_values back to the original size, filling with False and NaN, respectively
    rejected_full = np.full_like(p_values, False, dtype=bool)
    rejected_full[non_nan_indices] = rejected

    corrected_p_values_full = np.full_like(p_values, np.nan)
    corrected_p_values_full[non_nan_indices] = corrected_p_values

    # Find the significant indices
    significant_indices = [(i, j) for k, (i, j) in enumerate([(r, c) for r in range(rows) for c in range(columns)]) if rejected_full[k]]

    return significant_indices


def calc_interaction_metrics(three_d_array, index, alpha=0.05):
    rows, columns, _ = three_d_array.shape
    p_values = []
    mean_values = []
    std_values = []
    q1_values = []
    q2_values = []
    q3_values = []
    i_values = []
    j_values = []

    # Perform one-sample t-tests and collect statistics
    for i in range(rows):
        for j in range(columns):
            samples = three_d_array[i, j]
            _, p_value = stats.ttest_1samp(samples, 0)
            p_values.append(p_value)
            
            mean = np.mean(samples)
            mean_values.append(mean)
            
            std_dev = np.std(samples, ddof=1)
            std_values.append(std_dev)
            
            quantile_25 = np.percentile(samples, 25)
            q1_values.append(quantile_25)
            
            quantile_50 = np.percentile(samples, 50)
            q2_values.append(quantile_50)
            
            quantile_75 = np.percentile(samples, 75)
            q3_values.append(quantile_75)
            
            i_values.append(i)
            j_values.append(j)


    # Create a dataframe
    interaction_metrics_df = pd.DataFrame({
        'i': i_values,
        'j': j_values,
        'mean': mean_values,
        'std_dev': std_values,
        'Q1': q1_values,
        'median': q2_values,
        'Q3': q3_values,
        'p_values': p_values,
    })
    
    interaction_metrics_df[['affector', 'target']] = [(index[val1], index[val2]) for val1, val2 in 
                                                      zip(interaction_metrics_df['j'], interaction_metrics_df['i'])]
    
    valid_metrics_df = interaction_metrics_df.dropna().copy()

    rejected, corrected_p_values = smt.multipletests(valid_metrics_df['p_values'].values, alpha=1e-12, method='fdr_bh')[:2]
    valid_metrics_df['q_values'] = corrected_p_values
    final_df = valid_metrics_df[['affector', 'target', 'mean', 'std_dev', 'Q1', 'median', 'Q3', 'p_values', 'q_values']]
    final_df[final_df['q_values'] < 1e-12]

    return final_df

In [13]:
# Usage example
folder_path = "/Users/feng626/workspace/data/PredPheno/cellbox/results/top1k/"
index, results = compile_W_to_3d_array(folder_path)


In [14]:
sign_int = find_significant_indices_fdr(results, alpha=1e-12)
sign_interactions = [(index[val2], index[val1]) for val1, val2 in sign_int]
sign_interactions_df = pd.DataFrame(sign_interactions, columns = ['affector', 'target'])
sign_interactions_df

,affector,target
0,P_Light(umol_photons_m-2_s-1),H6G84_RS01130
1,P_pilB::Tn5,H6G84_RS01130
2,P_Biofilm,H6G84_RS01130
3,H6G84_RS04295,H6G84_RS04855
4,P_rpaA,H6G84_RS04855
...,...,...
1609,H6G84_RS04295,58
1610,H6G84_RS02410,58
1611,H6G84_RS13700,58
1612,H6G84_RS01645,58


In [15]:
metrics_df = calc_interaction_metrics(results, index, alpha=1e-12)

In [16]:
metrics_df

,affector,target,mean,std_dev,Q1,median,Q3,p_values,q_values
1,H6G84_RS04855,H6G84_RS01130,-0.000152,0.004551,-0.000149,-2.996978e-08,0.000067,2.899429e-01,3.909623e-01
2,H6G84_RS00870,H6G84_RS01130,-0.000007,0.003614,-0.000161,1.236027e-07,0.000135,9.505394e-01,9.651654e-01
3,H6G84_RS08385,H6G84_RS01130,-0.000184,0.009983,-0.000051,4.246052e-07,0.000166,5.611748e-01,6.529480e-01
4,H6G84_RS09745,H6G84_RS01130,-0.000966,0.012772,-0.000211,-3.160925e-07,0.000058,1.692222e-02,3.855307e-02
5,H6G84_RS01795,H6G84_RS01130,-0.000498,0.011530,-0.000383,-8.889314e-07,0.000128,1.725616e-01,2.584411e-01
...,...,...,...,...,...,...,...,...,...
51967,H6G84_RS01390,58,-0.007879,0.102781,-0.012109,-1.649578e-04,0.007839,1.551846e-02,3.586235e-02
51968,H6G84_RS08975,58,-0.015527,0.126214,-0.012161,-2.811186e-05,0.009619,1.068090e-04,4.738684e-04
51969,H6G84_RS06645,58,-0.002465,0.182317,-0.024682,-7.119133e-04,0.007048,6.690388e-01,7.461709e-01
51970,H6G84_RS06185,58,-0.015921,0.174820,-0.017441,-2.948390e-04,0.011527,4.063186e-03,1.143312e-02


In [17]:
#len(corrected_p_values[corrected_p_values < 1e-12])
len(metrics_df[metrics_df['q_values'] < 1e-12])

1614

In [18]:
metrics_df.to_csv('./data/transcriptome/Cellbox_res/top1k_summary.csv', index=False)